In [14]:
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
from scipy.interpolate import BSpline
from cmdstanpy import CmdStanModel
import arviz as az
import pandas as pd
from glob import glob
from sklearn import preprocessing
import csv
from scipy.interpolate import InterpolatedUnivariateSpline

In [15]:
def load_signals (files):
    signals  = np.zeros((len(files),), dtype = object)
    for index,i in enumerate(files):
        file = np.loadtxt(i, dtype = 'str' , unpack = True)
        file = np.char.replace(file,',','.')
        file = file.astype(np.float64)
        signals[index] = file
    return signals

def filtr_1(signal):
    index = 0
    y = 0

   
    while (y < 0.01):
        y = signal[1][index]
        index += 1

    index -= 2
    signal_y = signal[1][index:]
    signal_x = signal[0][index:]
    trunc_val_peak = -2.5
    signal_y = signal_y[signal_y > trunc_val_peak]

    index = 1
    while abs(signal_y[-index] - signal_y[-(index+1)]) < 0.02:
        index += 1

    signal_y = signal_y[:-index]
    signal_x = signal_x[:len(signal_y)]

    return signal_x,signal_y

def filtr_2(signal_x, signal_y):
    
    cum_sum = np.cumsum(signal_y[::-1])[::-1]
    cum_mean = np.flip(cum_sum) / np.arange(1,len(cum_sum)+1)

    index = 1
    while abs(signal_y[-index-1] - cum_mean[index]) < 0.02:
        index += 1
    
    index -= 2
    signal_y_filtered = signal_y[:-index]
    signal_x_filtered = signal_x[:len(signal_y_filtered)]
    
    return signal_x_filtered,signal_y_filtered

def rms(y,x):
    rms = []
    for i in range(len(x)-1):
        rms = np.append(rms,(np.trapz(np.square(y[i:2+i]),x[i:2+i])))
    rms = np.sqrt(np.sum(rms)/(max(x)- min(x)))
    return rms
    

    
def rms_signals(signals_x,signals_y):
        num_per_rms = 23
        num = np.ceil(len(signals_y)/num_per_rms)
        y = np.array_split(signals_y,num)
        time = np.array_split(signals_x,num)
        signal_rms = []
        time_rms = []
        for i in range(len(y)):
            signal_rms =np.append(signal_rms,rms(y[i],time[i]))
            time_rms = np.append(time_rms, np.mean(time[i]))
    
        return time_rms, signal_rms



min_max_scaler = preprocessing.MinMaxScaler()

In [16]:
files_healthy = glob('data_2/p0_*.*')
files_damaged_1 = glob('data_2/p1_*.*')
files_damaged_2 = glob('data_2/p2_*.*')
files_damaged_ring = glob('data_2/pi_*.*')

signals_healthy = load_signals(files_healthy)
signals_damaged_1 = load_signals(files_damaged_1)
signals_damaged_2 = load_signals(files_damaged_2)
signals_damaged_ring = load_signals(files_damaged_ring)

In [17]:
signals_healthy_x = np.zeros((len(signals_healthy),), dtype = object)
signals_healthy_y = np.zeros((len(signals_healthy),), dtype = object)

for index,i in enumerate(signals_healthy):
    signals_healthy_x[index],signals_healthy_y[index] = filtr_1(i)
    signals_healthy_x[index],signals_healthy_y[index] = rms_signals(signals_healthy_x[index],signals_healthy_y[index]) 
    signals_healthy_x[index],signals_healthy_y[index] = filtr_2(signals_healthy_x[index],signals_healthy_y[index])
    signals_healthy_x[index] = min_max_scaler.fit_transform(signals_healthy_x[index].reshape(-1, 1)).astype(float)    #normalization
    signals_healthy_y[index] = min_max_scaler.fit_transform(signals_healthy_y[index].reshape(-1, 1)).astype(float)    #normalization
    signals_healthy_x[index] = signals_healthy_x[index] - min(signals_healthy_x[index])

In [18]:
signals_damaged_1_x = np.zeros((len(signals_damaged_1),), dtype = object)
signals_damaged_1_y = np.zeros((len(signals_damaged_1),), dtype = object)

for index,i in enumerate(signals_damaged_1):
    signals_damaged_1_x[index],signals_damaged_1_y[index] = filtr_1(i)
    signals_damaged_1_x[index],signals_damaged_1_y[index] = rms_signals(signals_damaged_1_x[index],signals_damaged_1_y[index]) 
    signals_damaged_1_x[index],signals_damaged_1_y[index] = filtr_2(signals_damaged_1_x[index],signals_damaged_1_y[index])
    signals_damaged_1_x[index] = min_max_scaler.fit_transform(signals_damaged_1_x[index].reshape(-1, 1)).astype(float)    #normalization        
    signals_damaged_1_y[index] = min_max_scaler.fit_transform(signals_damaged_1_y[index].reshape(-1, 1)).astype(float)    #normalization    
    signals_damaged_1_x[index] = signals_damaged_1_x[index] - min(signals_damaged_1_x[index])

In [19]:
signals_damaged_2_x = np.zeros((len(signals_damaged_2),), dtype = object)
signals_damaged_2_y = np.zeros((len(signals_damaged_2),), dtype = object)

for index,i in enumerate(signals_damaged_2):
    signals_damaged_2_x[index],signals_damaged_2_y[index] = filtr_1(i)
    signals_damaged_2_x[index],signals_damaged_2_y[index] = rms_signals(signals_damaged_2_x[index],signals_damaged_2_y[index]) 
    signals_damaged_2_x[index],signals_damaged_2_y[index] = filtr_2(signals_damaged_2_x[index],signals_damaged_2_y[index])
    signals_damaged_2_x[index] = min_max_scaler.fit_transform(signals_damaged_2_x[index].reshape(-1, 1)).astype(float)    #normalization
    signals_damaged_2_y[index] = min_max_scaler.fit_transform(signals_damaged_2_y[index].reshape(-1, 1)).astype(float)    #normalization
    signals_damaged_2_x[index] = signals_damaged_2_x[index] - min(signals_damaged_2_x[index])


In [20]:
signals_damaged_ring_x = np.zeros((len(signals_damaged_ring),), dtype = object)
signals_damaged_ring_y = np.zeros((len(signals_damaged_ring),), dtype = object)

for index,i in enumerate(signals_damaged_ring):
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = filtr_1(i)
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = rms_signals(signals_damaged_ring_x[index],signals_damaged_ring_y[index]) 
    signals_damaged_ring_x[index],signals_damaged_ring_y[index] = filtr_2(signals_damaged_ring_x[index],signals_damaged_ring_y[index])
    signals_damaged_ring_x[index] = min_max_scaler.fit_transform(signals_damaged_ring_x[index].reshape(-1, 1)).astype(float)  #normalization
    signals_damaged_ring_y[index] = min_max_scaler.fit_transform(signals_damaged_ring_y[index].reshape(-1, 1)).astype(float)  #normalization
    signals_damaged_ring_x[index] = signals_damaged_ring_x[index] - min(signals_damaged_ring_x[index])
   


In [21]:
signals_damaged_1_res_y = np.zeros((len(signals_damaged_ring),), dtype = object)
signals_damaged_2_res_y = np.zeros((len(signals_damaged_ring),), dtype = object)
signals_damaged_ring_res_y = np.zeros((len(signals_damaged_ring),), dtype = object)


for index in range(len(signals_healthy_y)):
    x = signals_healthy_x[index]
    y = signals_healthy_y[index]
    s = InterpolatedUnivariateSpline(x, y)
    signals_damaged_1_res_y[index] = signals_damaged_1_y[index] - s(signals_damaged_1_x[index])
    signals_damaged_2_res_y[index] = signals_damaged_2_y[index] - s(signals_damaged_2_x[index])
    signals_damaged_ring_res_y[index] = signals_damaged_ring_y[index] - s(signals_damaged_ring_x[index])

In [37]:
for i in range(len(signals_healthy_y)):
    signals_healthy_y[i] = signals_healthy_y[i].reshape(len(signals_healthy_y[i]))
    signals_healthy_x[i] = signals_healthy_x[i].reshape(len(signals_healthy_x[i]))


for i in range(len(signals_damaged_1_res_y)):
    signals_damaged_1_res_y[i] = signals_damaged_1_res_y[i].reshape(len(signals_damaged_1_res_y[i]))
    signals_damaged_1_x[i] = signals_damaged_1_x[i].reshape(len(signals_damaged_1_res_y[i]))

for i in range(len(signals_damaged_2_res_y)):
    signals_damaged_2_res_y[i] = signals_damaged_2_res_y[i].reshape(len(signals_damaged_2_res_y[i]))
    signals_damaged_2_x[i] = signals_damaged_2_x[i].reshape(len(signals_damaged_2_x[i]))

for i in range(len(signals_damaged_ring_res_y)):
    signals_damaged_ring_res_y[i] = signals_damaged_ring_res_y[i].reshape(len(signals_damaged_ring_res_y[i]))
    signals_damaged_ring_x[i] = signals_damaged_ring_x[i].reshape(len(signals_damaged_ring_x[i]))

In [40]:
for i in range(len(signals_healthy)):
    data = pd.DataFrame((signals_healthy_y[i],signals_healthy_x[i]))
    data.to_csv('processed_data2/signals_healthy_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_1)):
    data = pd.DataFrame((signals_damaged_1_res_y[i],signals_damaged_1_x[i]))
    data.to_csv('processed_data2/signals_damaged_1_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_2)):
    data = pd.DataFrame((signals_damaged_2_res_y[i],signals_damaged_2_x[i]))
    data.to_csv('processed_data2/signals_damaged_2_{}.csv'.format(i), index=False)

for i in range(len(signals_damaged_ring)):
    data = pd.DataFrame((signals_damaged_ring_res_y[i],signals_damaged_ring_x[i]))
    data.to_csv('processed_data2/signals_damaged_ring_{}.csv'.format(i), index=False)